In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import matplotlib.ticker as mticker
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib import colors as mcolors
from matplotlib import legend_handler
from mycolorpy import colorlist as mcp
import func_timeout
import pomegranate
import networkx as nx
import pickle as p
import geopandas as gpd
from shapely.geometry import Polygon, Point
import rtree
import pygeos
import cloudpickle as cp

url = 'https://raw.githubusercontent.com/CrmnCA/inferring-urban-polycentricity-from-variability-in-human-mobility/master/'
import httpimport
with httpimport.remote_repo(["urban_structure_human_mobility"], url):
    import urban_structure_human_mobility as ushm

In [ ]:
def nucleus_list(gdf_distribution, main_nucleus, buffer_size):    
    gdf_distribution = gdf_distribution[gdf_distribution['no_journeys'] > 0]
    gdf_main_nucleus = gdf_distribution[gdf_distribution['station_name'] == main_nucleus].reset_index(drop=True)
    gdf_buffer_main_nucleus = gpd.GeoDataFrame({'geometry': [gdf_main_nucleus['geometry'].buffer(buffer_size)[0]]}, crs='3857')
    gdf_nucleus_list = gdf_distribution.overlay(gdf_buffer_main_nucleus, how='intersection').reset_index(drop=True)
    gdf_nucleus_list['distance_to_main_nucleus'] = [gdf_main_nucleus['geometry'][0].distance(gdf_nucleus_list.loc[i,'geometry']) for i in range(len(gdf_nucleus_list))]
    gdf_nucleus_list = gdf_nucleus_list.sort_values(by='distance_to_main_nucleus').reset_index(drop=True)
    return gdf_nucleus_list


In [ ]:
def distiribution_network(gdf_distribution, G, nucleus_node):
    network_distance_to_nucleus = []
    for i in range(len(gdf_distribution)):
        network_distance_to_nucleus.append(len(nx.shortest_path(G, gdf_distribution.loc[i, 'node_id'], nucleus_node))-1)
    gdf_distribution['network_distance_to_nucleus'] = network_distance_to_nucleus
    return gdf_distribution  
    

In [ ]:
city = 'Seoul'
gdf_distribution = gpd.read_file(url + str(city) + '_distribution.gpkg').to_crs('3857')
G = pd.read_pickle(url + str(city) + '_Network.p')
if city == 'London':
    main_nucleus = 'Piccadilly Circus' 
else:
    main_nucleus = 'City Hall Station (Seoul)' 
buffer_size=3000
gdf_nucleus_list = nucleus_list(gdf_distribution, main_nucleus, buffer_size)

In [ ]:
# Initialise databases to store results of analysis

df_results_monocentric_hypothesis = pd.DataFrame(columns = ['nucleus_name', 'node_id', 
                                                'intercept', 'slope', 'intercept_se', 'slope_se', 'r', 'r_pvalue'])
df_results_model_2mix = pd.DataFrame(columns = ['nucleus_name', 'node_id',
                                                'intercept_proximal', 'slope_proximal', 'intercept_se_proximal', 'slope_se_proximal', 'r_proximal', 'r_pvalue_proximal',
                                                'intercept_distal', 'slope_distal', 'intercept_se_distal', 'slope_se_distal', 'r_distal', 'r_pvalue_distal',
                                                'loglikelihood', 'no_parameters', 'dof', 'BIC'
                                               ])
df_results_model_3mix = pd.DataFrame(columns = ['nucleus_name', 'node_id',
                                                'intercept_proximal', 'slope_proximal', 'intercept_se_proximal', 'slope_se_proximal', 'r_proximal', 'r_pvalue_proximal',
                                                'intercept_medial', 'slope_medial', 'intercept_se_medial', 'slope_se_medial', 'r_medial', 'r_pvalue_medial',
                                                'intercept_distal', 'slope_distal', 'intercept_se_distal', 'slope_se_distal', 'r_distal', 'r_pvalue_distal',
                                                'loglikelihood', 'no_parameters', 'dof', 'BIC'
                                               ])



In [ ]:
# Compute results of analysis for different choices of nucleus within a distance (in meters) given by buffer_size from the main nucleus

for i in range(len(gdf_nucleus_list)):
    
    print(i, 'Nucleus: ' + str(gdf_nucleus_list.loc[i, 'station_name']))
    
    nucleus_node = gdf_nucleus_list.loc[i, 'node_id']
    gdf_distribution_network = distiribution_network(gdf_distribution, G, nucleus_node)
    gdf_distribution_network = gdf_distribution_network[gdf_distribution_network['average_length_journeys'] > 0].reset_index(drop=True)
    df_distribution_network = pd.DataFrame(gdf_distribution_network.drop(columns='geometry'))
    
    results_monocentric_hypothesis = ushm.monocentric_hypothesis(df_distribution_network, i, city)
    df_results_monocentric_hypothesis_nucleus = pd.DataFrame([[gdf_nucleus_list.loc[i, 'station_name'], nucleus_node] +
                      results_monocentric_hypothesis],
                      columns=df_results_monocentric_hypothesis.columns)
    df_results_monocentric_hypothesis = pd.concat([df_results_monocentric_hypothesis, df_results_monocentric_hypothesis_nucleus], ignore_index=True)
    
    results_model_2mix = ushm.model_2mix(df_distribution_network, i, city)
    df_results_model_2mix_nucleus = pd.DataFrame([[gdf_nucleus_list.loc[i, 'station_name'], nucleus_node] +
                      results_model_2mix],
                      columns=df_results_model_2mix.columns)
    df_results_model_2mix = pd.concat([df_results_model_2mix, df_results_model_2mix_nucleus], ignore_index=True)

    results_model_3mix = ushm.model_3mix(df_distribution_network, i, city)
    df_results_model_3mix_nucleus = pd.DataFrame([[gdf_nucleus_list.loc[i, 'station_name'], nucleus_node] +
                      results_model_3mix],
                      columns=df_results_model_3mix.columns)
    df_results_model_3mix = pd.concat([df_results_model_3mix, df_results_model_3mix_nucleus], ignore_index=True)
    
print('\n\nMonocentric hypothesis.' +
    '\n\nFor the main nucleus, ' + str(df_results_monocentric_hypothesis.loc[0, 'nucleus_name']) + ', we obtain the following regression line: ' +
    '\nIntercept = ' + str(round(df_results_monocentric_hypothesis.loc[0, 'intercept'],3)) + ' ± ' + str(round(df_results_monocentric_hypothesis.loc[0, 'intercept_se'],3)) +
    '\nSlope = ' + str(round(df_results_monocentric_hypothesis.loc[0, 'slope'],3)) + ' ± ' + str(round(df_results_monocentric_hypothesis.loc[0, 'slope_se'],3)) +
    '\nR = ' + str(round(df_results_monocentric_hypothesis.loc[0, 'r'],3)) + ' with p-value = ' + str(round(df_results_monocentric_hypothesis.loc[0, 'r_pvalue'],3)) +                                                                                               
    '\n\nRelative standard deviation of regression line parameters obtained by considering different nuclei within a distance of ' + str(buffer_size/1000) + ' km from the main nucleus is:' +
    '\nIntercept = ' + str(round(np.std(df_results_monocentric_hypothesis['intercept'])/np.mean(df_results_monocentric_hypothesis['intercept'])*100, 3)) + '%' + 
    '\nSlope = ' + str(round(np.std(df_results_monocentric_hypothesis['slope'])/np.mean(df_results_monocentric_hypothesis['slope'])*100, 3)) + '%' +
    '\nNote: the relative standard deviation of a regression line parameter is computed with respect to the average across the regression line parameter corresponding to each choice of nucleus.')
 
print('\n\nTwo-component mixture model.' +
    '\n\nFor the main nucleus, ' + str(df_results_model_2mix.loc[0, 'nucleus_name']) + ', we obtain the following regression lines: ' +
    '\nIntercept proximal = ' + str(round(df_results_model_2mix.loc[0, 'intercept_proximal'],3)) + ' ± ' + str(round(df_results_model_2mix.loc[0, 'intercept_se_proximal'],3)) +
    '\nSlope proximal = ' + str(round(df_results_model_2mix.loc[0, 'slope_proximal'],3)) + ' ± ' + str(round(df_results_model_2mix.loc[0, 'slope_se_proximal'],3)) +
    '\nR proximal = ' + str(round(df_results_model_2mix.loc[0, 'r_proximal'],3)) + ' with p-value = ' + str(round(df_results_model_2mix.loc[0, 'r_pvalue_proximal'],3)) +                                                                                               
    '\nIntercept distal = ' + str(round(df_results_model_2mix.loc[0, 'intercept_distal'],3)) + ' ± ' + str(round(df_results_model_2mix.loc[0, 'intercept_se_distal'],3)) +
    '\nSlope distal = ' + str(round(df_results_model_2mix.loc[0, 'slope_distal'],3)) + ' ± ' + str(round(df_results_model_2mix.loc[0, 'slope_se_distal'],3)) +
    '\nR distal = ' + str(round(df_results_model_2mix.loc[0, 'r_distal'],3)) + ' with p-value = ' + str(round(df_results_model_2mix.loc[0, 'r_pvalue_distal'],3)) +                                                                                               
    '\n\nRelative standard deviation of regression line parameters obtained by considering different nuclei within a distance of ' + str(buffer_size/1000) + ' km from the main nucleus is:' +
    '\nIntercept proximal = ' + str(round(np.std(df_results_model_2mix['intercept_proximal'])/np.mean(df_results_model_2mix['intercept_proximal'])*100, 3)) + '%' + 
    '\nSlope proximal = ' + str(round(np.std(df_results_model_2mix['slope_proximal'])/np.mean(df_results_model_2mix['slope_proximal'])*100, 3)) + '%' +
    '\nIntercept distal = ' + str(round(np.std(df_results_model_2mix['intercept_distal'])/np.mean(df_results_model_2mix['intercept_distal'])*100, 3)) + '%' + 
    '\nSlope distal = ' + str(round(np.std(df_results_model_2mix['slope_distal'])/np.mean(df_results_model_2mix['slope_distal'])*100, 3)) + '%' +
    '\nNote: the relative standard deviation of a regression line parameter is computed with respect to the average across the regression line parameter corresponding to each choice of nucleus.')

print('\n\nThree-component mixture model.' +
    '\n\nFor the main nucleus, ' + str(df_results_model_3mix.loc[0, 'nucleus_name']) + ', we obtain the following regression lines: ' +
    '\nIntercept proximal = ' + str(round(df_results_model_3mix.loc[0, 'intercept_proximal'],3)) + ' ± ' + str(round(df_results_model_3mix.loc[0, 'intercept_se_proximal'],3)) +
    '\nSlope proximal = ' + str(round(df_results_model_3mix.loc[0, 'slope_proximal'],3)) + ' ± ' + str(round(df_results_model_3mix.loc[0, 'slope_se_proximal'],3)) +
    '\nR proximal = ' + str(round(df_results_model_3mix.loc[0, 'r_proximal'],3)) + ' with p-value = ' + str(round(df_results_model_3mix.loc[0, 'r_pvalue_proximal'],3)) +                                                                                               
    '\nIntercept medial = ' + str(round(df_results_model_3mix.loc[0, 'intercept_medial'],3)) + ' ± ' + str(round(df_results_model_3mix.loc[0, 'intercept_se_medial'],3)) +
    '\nSlope medial = ' + str(round(df_results_model_3mix.loc[0, 'slope_medial'],3)) + ' ± ' + str(round(df_results_model_3mix.loc[0, 'slope_se_medial'],3)) +
    '\nR medial = ' + str(round(df_results_model_3mix.loc[0, 'r_medial'],3)) + ' with p-value = ' + str(round(df_results_model_3mix.loc[0, 'r_pvalue_medial'],3)) +                                                                                               
    '\nIntercept distal = ' + str(round(df_results_model_3mix.loc[0, 'intercept_distal'],3)) + ' ± ' + str(round(df_results_model_3mix.loc[0, 'intercept_se_distal'],3)) +
    '\nSlope distal = ' + str(round(df_results_model_3mix.loc[0, 'slope_distal'],3)) + ' ± ' + str(round(df_results_model_3mix.loc[0, 'slope_se_distal'],3)) +
    '\nR distal = ' + str(round(df_results_model_3mix.loc[0, 'r_distal'],3)) + ' with p-value = ' + str(round(df_results_model_3mix.loc[0, 'r_pvalue_distal'],3)) +                                                                                               
    '\n\nRelative standard deviation of regression line parameters obtained by considering different nuclei within a distance of ' + str(buffer_size/1000) + ' km from the main nucleus is:' +
    '\nIntercept proximal = ' + str(round(np.std(df_results_model_3mix['intercept_proximal'])/np.mean(df_results_model_3mix['intercept_proximal'])*100, 3)) + '%' + 
    '\nSlope proximal = ' + str(round(np.std(df_results_model_3mix['slope_proximal'])/np.mean(df_results_model_3mix['slope_proximal'])*100, 3)) + '%' +
    '\nIntercept medial = ' + str(round(np.std(df_results_model_3mix['intercept_medial'])/np.mean(df_results_model_3mix['intercept_medial'])*100, 3)) + '%' + 
    '\nSlope medial = ' + str(round(np.std(df_results_model_3mix['slope_medial'])/np.mean(df_results_model_3mix['slope_medial'])*100, 3)) + '%' +
    '\nIntercept distal = ' + str(round(np.std(df_results_model_3mix['intercept_distal'])/np.mean(df_results_model_3mix['intercept_distal'])*100, 3)) + '%' + 
    '\nSlope distal = ' + str(round(np.std(df_results_model_3mix['slope_distal'])/np.mean(df_results_model_3mix['slope_distal'])*100, 3)) + '%' +
    '\nNote: the relative standard deviation of a regression line parameter is computed with respect to the average across the regression line parameter corresponding to each choice of nucleus.')
 
 

In [ ]:
print(df_results_monocentric_hypothesis)

In [ ]:
print(df_results_model_2mix)

In [ ]:
print(df_results_model_3mix)